In [1]:
import numpy as np
import pandas as pd
from typing import List
from sklearn.metrics.pairwise import cosine_similarity


In [2]:
import numpy as np
import pandas as pd

# さっきコピーしたファイルを読み込み
embeddings = np.load("embeddings_all-MiniLM-L6-v2.npy")
chunks = pd.read_parquet("chunks_all-MiniLM-L6-v2.parquet")

print("embeddings shape:", embeddings.shape)
print("chunks shape:", chunks.shape)
chunks.head()


embeddings shape: (12239, 384)
chunks shape: (12239, 6)


,doc_idx,chunk_id,chunk_text,id,midashi_1,kaisai_date
0,0,0,１．調査目的と調査概要スライド番号３スライド番号４４．スケジュール（案）（参考）ｗｇコメント...,26317,東京電力(株)等との面談(平成25年3月),平成25年3月4日
1,0,1,た漏水は確認されなかった①②③漏水は確認されなかったベント管スリーブ端部サンドクッションドレ...,26317,東京電力(株)等との面談(平成25年3月),平成25年3月4日
2,1,0,原子力発電所の新規制基準適合性に係る審査会合（第１回）１．日時：平成２５年７月１６日（火）９...,34305,第1回原子力発電所の新規制基準適合性に係る審査会合,平成25年7月16日
3,2,0,原子力発電所の新規制基準適合性に係る審査会合（第２回）１．日時：平成２５年７月２３日（火）１...,34313,第2回原子力発電所の新規制基準適合性に係る審査会合,平成25年7月23日
4,3,0,原子力発電所の新規制基準適合性に係る審査会合（第３回）１．日時：平成２５年７月２５日（木）１...,34321,第3回原子力発電所の新規制基準適合性に係る審査会合,平成25年7月25日


In [3]:
from pathlib import Path
import numpy as np
import pandas as pd

def load_index():
    chunks = pd.read_parquet("chunks_all-MiniLM-L6-v2.parquet")
    embeddings = np.load("embeddings_all-MiniLM-L6-v2.npy")
    return chunks, embeddings

df_chunks_loaded, embs_loaded = load_index()


質問文シソーラス機能

In [4]:
import re

def apply_thesaurus(text: str) -> str:
    """
    質問文などの日本語テキストに対し、シソーラス変換を適用。
    - 伊方発電所3号/３号 → 伊方発電所3号機/３号機（既に号機なら何もしない）
    - ルールを増やしたい場合は PATTERNS に (pattern, repl) を追加
    """
    if not isinstance(text, str) or not text:
        return text

    # ここに置換ルールを追加していく（上から順に適用）
    PATTERNS = [
        # 伊方発電所 + (3|３) + 号  →  … 号機
        # すでに「号機」になっていたら置換しないために (?!機) を入れる
        (
            r"(伊方発電所)\s*((?:3|３))\s*号(?!機)",
            r"\1\2号機",
        ),
        # 必要なら別号機や別原発もここにルール追加
        # 例: 川内原発(1|１)号 → 川内原発(1|１)号機
        # (r"(川内原発)\s*((?:1|１))\s*号(?!機)", r"\1\2号機"),
    ]

    out = text
    for pat, repl in PATTERNS:
        out = re.sub(pat, repl, out)

    return out

HyDE機能

In [5]:
import requests
from typing import Optional, Tuple

# ====== 設定 ======
MIN_LEN = 20  # 文字数しきい値

# LLM（LM Studio）の OpenAI互換API
LLM_URL = "http://127.0.0.1:1234/v1/chat/completions"
LLM_MODEL = "qwen/qwen3-vl-8b"
# ==================


def count_chars(text: str) -> int:
    """Pythonのlen()で文字数カウント（全角/半角ともに1カウント）。"""
    if not isinstance(text, str):
        return 0
    return len(text)


def build_hyde_prompt(question: str) -> str:
    """HyDE 用のプロンプトを作成"""
    return (
        "あなたは質問を理解し、背景や前提、条件、具体例を補って、"
        "内容を自然に拡張するアシスタントです。\n\n"
        f"【質問】\n{question}\n\n"
        "【指示】\n"
        "・上の質問を、意味を変えずに50文字以上へ自然に拡張してください。\n"
        "・追加の条件、背景、具体例、目的、制約などを自然に加えてください。\n"
        "・出力は1つの長い質問文（または依頼文）のみを返してください。"
    )


def hyde_expand_with_local_llm(question: str) -> Optional[str]:
    """ローカル LLM（LM Studio）で HyDE 拡張"""
    try:
        prompt = build_hyde_prompt(question)

        payload = {
            "model": LLM_MODEL,
            "messages": [
                {"role": "system", "content": "あなたは丁寧で簡潔な日本語の文章作成アシスタントです。"},
                {"role": "user", "content": prompt},
            ],
            "temperature": 0.7,
        }

        resp = requests.post(LLM_URL, json=payload, timeout=60)
        resp.raise_for_status()
        data = resp.json()

        expanded = data["choices"][0]["message"]["content"].strip()
        return expanded

    except Exception as e:
        print("LLMjp 経由のHyDE拡張に失敗:", e)
        return None


def hyde_expand_fallback(question: str, min_len: int = MIN_LEN) -> str:
    """ローカルLLMが応答しないときのフォールバック"""
    base = question.strip()
    addon = "（背景や目的、前提条件、具体例を補足して明確化してください）"
    candidate = f"{base}。{addon}"

    if len(candidate) < min_len:
        need = min_len - len(candidate)
        candidate += " 詳細を含めて説明してください。" + ("。" * max(0, need - 12))

    return candidate


def maybe_hyde_expand(question: str, min_len: int = MIN_LEN) -> Tuple[str, int, bool]:
    """
    HyDE 拡張本体
    - 文字数50以上 → 何もしない
    - 文字数50未満 → ローカルLLMで拡張（失敗時フォールバック）
    """
    n = count_chars(question)

    if n >= min_len:
        return question, n, False

    expanded = hyde_expand_with_local_llm(question)
    if not expanded:
        expanded = hyde_expand_fallback(question, min_len=min_len)

    return expanded, count_chars(expanded), True


def preprocess_question_with_hyde_then_tokens(question: str):
    """他処理につなぐ HyDE 拡張→質問返却"""
    expanded_question, _, _ = maybe_hyde_expand(question, min_len=MIN_LEN)
    return expanded_question


In [6]:
import requests


def llm_chat(messages, max_tokens=1024, temperature=0.2):
    payload = {
        "model": LLM_MODEL,
        "messages": messages,
        "max_tokens": max_tokens,
        "temperature": temperature,
    }

    resp = requests.post(LLM_URL, json=payload)
    resp.raise_for_status()
    return resp.json()["choices"][0]["message"]["content"]

# テスト
print(llm_chat([{"role": "user", "content": "ローカル LLM 接続テストです。"}]))


こんにちは！ローカル LLM 接続テストを行いますね 😊

**テスト内容：**
- ローカル環境で LLM が正常に動作しているか確認
- リクエストが正しく処理され、レスポンスが返ってくるか確認

**テスト結果：**
✅ 接続成功！  
✅ リクエスト受信成功  
✅ 正常なレスポンス返却

**テストメッセージ：**  
「ローカル LLM は正常に動作しています！」

---

何か特定のタスクや質問があれば、いつでもお知らせください。  
例：  
- 「この文章を要約して」  
- 「この質問に答えなさい」  
- 「ローカル LLM の設定を確認したい」

お気軽にどうぞ！ 🚀


キーワード抽出

In [7]:
# from janome.tokenizer import Tokenizer
# from typing import List
# import numpy as np
# from sklearn.feature_extraction.text import TfidfVectorizer

# # --- トークナイズ：名詞＋連続名詞の複合語（n>=2 は "_" 連結） ---
# _t = Tokenizer()

# def extract_nouns_and_compounds(text: str) -> List[str]:
#     if not isinstance(text, str) or not text.strip():
#         return []
#     nouns, spans, cur = [], [], []
#     for tok in _t.tokenize(text):
#         base = tok.base_form if tok.base_form != "*" else tok.surface
#         pos0 = tok.part_of_speech.split(",")[0]
#         if pos0 == "名詞" and base:
#             nouns.append(base)
#             cur.append(base)
#         else:
#             if cur:
#                 spans.append(cur)
#                 cur = []
#     if cur:
#         spans.append(cur)

#     # 名詞が連続している部分から複合語を作る
#     compounds = []
#     for span in spans:
#         if len(span) >= 2:
#             for L in range(2, len(span) + 1):
#                 for i in range(len(span) - L + 1):
#                     compounds.append("_".join(span[i:i+L]))

#     return nouns + compounds


# # --- TF-IDFで上位キーワード抽出 ---
# def tfidf_top_k(tokens_list: List[List[str]], k: int = 5) -> List[str]:
#     vec = TfidfVectorizer(
#         analyzer="word",
#         tokenizer=lambda x: x,
#         preprocessor=lambda x: x,
#         token_pattern=None,
#         ngram_range=(1, 1),
#         min_df=1,
#     )
#     X = vec.fit_transform(tokens_list)
#     feats = np.array(vec.get_feature_names_out())

#     row = X.getrow(0)
#     if row.nnz == 0:
#         return []

#     idx = row.indices
#     data = row.data
#     top = idx[np.argsort(-data)[:k]]

#     return feats[top].tolist()


# def extract_keywords_tfidf(text: str, top_k: int = 5) -> List[str]:
#     tokens = extract_nouns_and_compounds(text)
#     if not tokens:
#         return []
#     return tfidf_top_k([tokens], k=top_k)


#LLMによるキーワード検索

In [8]:
def generate_keywords_llm(question: str, max_k: int = 3) -> List[str]:
    prompt = f"""
以下の質問文から重要キーワードのみ抽出してください。

【基準】
- 数字を含む語は必ず抽出
- 専門用語・固有名詞・珍しい語を優先
- 一般語（こと／ため／必要／対応／実施／もの／これ／それ等）は除外
- 長すぎる語（10字超）は分割して抽出
- 助詞・接続詞は除外
- 最大 {max_k} 個
- 出力は JSON 配列のみ（説明禁止）

【質問】
{question}
"""

    payload = {
        "model": LLM_MODEL,
        "messages": [
            {"role": "user", "content": prompt}
        ],
        "temperature": 0.2,
    }

    try:
        resp = requests.post(LLM_URL, json=payload)
        resp.raise_for_status()
        content = resp.json()["choices"][0]["message"]["content"].strip()

        # JSON 以外が混じっていたら除去する安全処理
        import json
        keywords = json.loads(content)

        # 念のため文字列だけにする
        keywords = [k for k in keywords if isinstance(k, str)]

        return keywords

    except Exception as e:
        print("LLM keyword generation failed:", e)
        return []


#or検索→and検索

In [9]:
#and検索
# import re
# import numpy as np

# def keyword_mask_on_chunks(df_chunks: pd.DataFrame, keywords):
#     if not keywords:
#         return np.ones(len(df_chunks), dtype=bool)

#     s = df_chunks["chunk_text"].fillna("")
#     mask = np.ones(len(df_chunks), dtype=bool)

#     # すべてのキーワードを含む行だけ True（AND 検索）
#     for kw in keywords:
#         mask &= s.str.contains(re.escape(kw), regex=True)

#     return mask

# or検索
import re
import numpy as np

def keyword_mask_on_chunks(df_chunks: pd.DataFrame, keywords):
    if not keywords:
        return np.ones(len(df_chunks), dtype=bool)
    pattern = "|".join(map(re.escape, keywords))
    return df_chunks["chunk_text"].fillna("").str.contains(pattern, regex=True).to_numpy()

In [10]:
import faiss
import pandas as pd

def build_faiss_ip(embs: np.ndarray):
    if embs.size == 0:
        return None
    d = embs.shape[1]
    index = faiss.IndexFlatIP(d)     # normalize_embeddings=True なので内積=コサイン
    index.add(embs.astype(np.float32))
    return index

def vector_search_in_filtered_chunks(df_chunks, embs, encoder, query, pre_mask=None, keywords=None, top_k=5):
    # どちらか指定：pre_mask優先
    if pre_mask is None:
        pre_mask = keyword_mask_on_chunks(df_chunks, keywords or [])
    if pre_mask.sum() == 0:
        return df_chunks.head(0).assign(score=[])

    df_f = df_chunks.loc[pre_mask].reset_index(drop=True)
    embs_f = embs[pre_mask]

    index = build_faiss_ip(embs_f)
    if index is None:
        return df_chunks.head(0).assign(score=[])

    q = encoder.encode([query], convert_to_numpy=True, normalize_embeddings=True).astype(np.float32)
    scores, idxs = index.search(q, top_k)
    scores, idxs = scores[0], idxs[0]

    hits = []
    for rank, (i_sub, s) in enumerate(zip(idxs, scores), 1):
        if i_sub < 0:
            continue
        row = df_f.iloc[int(i_sub)].copy()
        row["rank"] = rank
        row["score"] = float(s)
        hits.append(row)
    return pd.DataFrame(hits).sort_values("rank")

In [11]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer

EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2"

# ローカルのベクトル＆チャンクをロード
def load_index():
    df_chunks = pd.read_parquet("chunks_all-MiniLM-L6-v2.parquet")
    embs = np.load("embeddings_all-MiniLM-L6-v2.npy")
    encoder = SentenceTransformer(EMBEDDING_MODEL)
    print(f"📚 chunks={len(df_chunks)}, embs={embs.shape}")
    return df_chunks, embs, encoder

df_chunks, embs, encoder = load_index()


📚 chunks=12239, embs=(12239, 384)


In [12]:
def search(question: str, top_k: int = 0):
    # 1) 質問前処理
    q = apply_thesaurus(question)
    q = preprocess_question_with_hyde_then_tokens(q)



    #旧：TF-IDF
    #keywords = extract_keywords_tfidf(q, top_k=3)
    # 新：LLMベース
    keywords = generate_keywords_llm(q, max_k=3)
    #keywords = ["残留熱除去系"]


    print("📝 question:", question)
    print("📝 question (HyDE後):", q)
    print("🔑 keywords:", keywords)

    # 2) キーワードでチャンク絞り込み
    mask = keyword_mask_on_chunks(df_chunks, keywords)
    print(f"🎯 フィルタ後チャンク数: {mask.sum()} / {len(df_chunks)}")

    # 3) ベクトル検索
    results = vector_search_in_filtered_chunks(
        df_chunks=df_chunks,
        embs=embs,
        encoder=encoder,
        query=q,
        pre_mask=mask,
        top_k=top_k,
    )

    cols = ["rank", "score", "id", "midashi_1", "doc_idx", "chunk_id", "chunk_text", "kaisai_date"]
    return results[cols]

In [13]:
import textwrap

def build_context_from_results(results: pd.DataFrame, max_chars: int = 4000) -> str:
    """
    search() の結果 DataFrame から、LLM に渡すコンテキスト文字列を作る。
    長くなりすぎないように max_chars で打ち切る。
    """
    parts = []
    total_len = 0

    for _, row in results.iterrows():
        header = f"[doc_idx={row.get('doc_idx')}, chunk_id={row.get('chunk_id')}, title={row.get('title_1', '')}]"
        body = str(row["chunk_text"])
        block = header + "\n" + textwrap.fill(body, 80) + "\n"

        if total_len + len(block) > max_chars:
            break
        parts.append(block)
        total_len += len(block)

    if not parts:
        return "(コンテキストが見つかりませんでした)"

    return "\n---\n".join(parts)


def answer_with_rag(question: str, top_k: int = 5, max_context_chars: int = 4000):
    """
    ローカルLLM（LLMjp）を利用した RAG 回答生成。
    1. search() で関連チャンクを取得
    2. そのコンテキストを元に LLMjp で回答を生成
    """
    # 1) ベクトル検索
    results = search(question, top_k=top_k)

    if len(results) == 0:
        return "関連するコンテキストが見つかりませんでした。", results

    # 2) コンテキスト組み立て
    context = build_context_from_results(results, max_chars=max_context_chars)

    system_prompt = """あなたは日本語で回答する原子力・工学系の専門アシスタントです。
以下の「コンテキスト」に含まれる情報を最優先して使い、
コンテキストに書いていないことは憶測で補わず、
その場合は「資料からは読み取れません」と答えてください。"""

    user_prompt = f"""# 質問
{question}

# コンテキスト（検索結果）
{context}
"""

    # LLMjpを呼ぶ
    answer = llm_chat(
        [
            {"role": "system", "content": system_prompt},
            {"role": "user",   "content": user_prompt},
        ],
        max_tokens=1024,
        temperature=0.2,
    )

    return answer.strip(), results


In [30]:
q = """
ｊｒｒ－３の原子炉施設の設置申請にあたり、どの程度の地震を想定していますか。簡潔に教えて。
"""

answer, used_chunks = answer_with_rag(q, top_k=5)

print("▼ 回答\n")
print(answer)

print("\n▼ 参照されたチャンク（上位）\n")
display(used_chunks[["rank", "score", "id", "midashi_1", "doc_idx", "chunk_id", "kaisai_date"]].head())

📝 question: 
ｊｒｒ－３の原子炉施設の設置申請にあたり、どの程度の地震を想定していますか。簡潔に教えて。

📝 question (HyDE後): 
ｊｒｒ－３の原子炉施設の設置申請にあたり、どの程度の地震を想定していますか。簡潔に教えて。

🔑 keywords: ['jr', '3', '地震']
🎯 フィルタ後チャンク数: 3335 / 12239
▼ 回答

資料からは読み取れません。

▼ 参照されたチャンク（上位）



,rank,score,id,midashi_1,doc_idx,chunk_id,kaisai_date
3282,1,0.727680,37418,事業者とのヒアリング概要・資料,645,0,None
3284,2,0.712691,37427,事業者とのヒアリング概要・資料,649,0,平成26年08月19日
3254,3,0.691155,36763,平成25年11月22日,446,0,平成25年11月22日
2470,4,0.678531,35792,第118回原子力発電所の新規制基準適合性に係る審査会合,218,0,平成26年6月13日
2032,5,0.678223,35645,泊発電所 ３号炉 関連審査会合 平成25年度,182,127,平成26年2月18日


In [23]:
# used_chunks に doc_idx, chunk_id が入っている前提
inspect_df = (
    used_chunks[["rank", "score", "doc_idx", "chunk_id"]]
    .merge(
        chunks[["doc_idx", "chunk_id", "chunk_text"]],
        on=["doc_idx", "chunk_id"],
        how="left",
    )
)

for _, row in inspect_df.iterrows():
    print(f'---- rank {row["rank"]} (doc_idx={row["doc_idx"]}, chunk_id={row["chunk_id"]}) ----')
    print(row["chunk_text"])
    print()


---- rank 1 (doc_idx=645, chunk_id=0) ----
６．提出資料・浜岡原子力発電所４号炉敷地における地震動の増幅特性について－２－

---- rank 2 (doc_idx=649, chunk_id=0) ----
③中部電力から、本日の指摘等について了解した旨の回答があった。６．提出資料・浜岡原子力発電所４号炉敷地における地震動の増幅特性について（コメント回答）・防波堤等の各施設の設計における基準地震動の使い分けについて－２－

---- rank 3 (doc_idx=446, chunk_id=0) ----
点で取得されている観測記録等を用いて評価を進めること。③九州電力から、本日の指摘等について了解した旨の回答があった。６．提出資料・川内原子力発電所・玄海原子力発電所震源を特定せず策定する地震動について（第４４回審査会合でのコメントを踏まえた検討状況）－２－

---- rank 4 (doc_idx=218, chunk_id=0) ----
原子力発電所の新規制基準適合性に係る審査会合（第１１８回）１．日時平成２６年６月１３日（金）１３：３０～１７：３０２．場所原子力規制委員会１３ｆ会議室ａ３．議題（１）地震について（２）その他４．配付資料資料１高浜発電所３・４号機原子炉建屋他の基礎地盤及び周辺斜面の安定性評価について

---- rank 5 (doc_idx=182, chunk_id=127) ----
地震・津波関係の審査会合を開催する予定になっています。それでは、以上で本日の審議を終了いたします。１２４



In [24]:
target = chunks[
    (chunks["doc_idx"] == 103) & (chunks["chunk_id"] == 26)
]["chunk_text"].iloc[0]

print("---- target chunk ----")
print(target)


---- target chunk ----
ます。これも表になってございまして、７日間で使います燃料を全て合計してございます。欄の一番左側が移動式大容量発電機、それからその次の欄が使用済燃料ピット復水タンク補給用の水中ポンプ用の発電機、水中ポンプは、ポンプに電源をつないでやるんですけども、その電源は発電機車、いわゆる作業工事現場にありますようなポータブルな発電機を、四角くてちょっと大き目のやつですけども、配備しましてやります。その発電機に燃料が必要ですので、その燃料の量を計算しましたのがこの欄でございます。それから、移動式の大容量ポンプ車、それからその右側ですけども、先ほどは復水タンクの給水用の水中ポンプ、それから一番右側の欄が使用済燃料ピットへの給水用の水中ポンプで、これらの７日間で必要な油の量を合計しますと、合計の欄でございますけども、２６万６，６０４ｌでございまして、備蓄している油の量が５１万８，５００ｌということで、十分油の量があるということが確認できています。１号機、２号機とも同様でございます。それから、次の欄が……。○更田委員すみません、時間の関係があるので、１回ここで切らせてください。○九州電力（須藤）はい。○更田委員資料２－１、２－２も使いながらですけども、資料２－１でここまで説明していただいた点について、少し上からと思いますけど、重要事故シーケンスに関しては、１次系冷却材ポンプシール部からの冷却材喪失を伴うｓｂｏの際に１次冷却材の流出を伴うシーケンスについてですけれども、まずシナリオと最初の条件等々からだと思いますけども、質問が２７あれば。○浦野調整官安全規制調整官、川内原子力発電所１・２号機を担当しています、浦野でございます。条件のところでございますけれども、ただいま御説明の５ページのところの炉心崩壊熱ということで、ここでは条件設定の考え方のみの御説明にちょっととどまっておりまして、実際にどのようなタイプの燃料が装荷されておって、それが取替炉心としてどのような組み合わせになっている、それが末期となっている値としてどのような値となっているのかと、これは後に出てきます発生する崩壊熱と熱除去のバランスのグラフがございますけれども、そういったとことも関係しますので、具体的な説明はぜひしていただきたいと思うのですが、いかがでしょうか。○九州電力（畠埜）発